In [ ]:
CREATE TABLE ecommerce_raw_data (
	order_id TEXT,  
	order_item_id TEXT,
	customer_id TEXT,  
	product_id TEXT,  
	campaign_id TEXT, 
	campaign_type TEXT, 
	channel_id TEXT,  
	platform_id TEXT,  
	date TEXT,  
	customer_name TEXT,  
	email TEXT,  
	customer_type TEXT,  
	country TEXT,  
	product_name TEXT,  
	category TEXT,  
	item_price TEXT,   
	quantity TEXT,  
	channel_name TEXT,  
	platform_name TEXT,
	impressions TEXT,  
	spend TEXT,  
	clicks TEXT,  
	orders TEXT,  
	revenue TEXT
);

In [ ]:
COPY ecommerce_raw_data
FROM 'C:/Program Files/PostgreSQL/17/ecommerce.csv'
DELIMITER ','
CSV HEADER;

---

In [ ]:
CREATE TABLE fact_orders (
	order_id VARCHAR,
	order_date_id INT,
	customer_id VARCHAR,
	channel_id VARCHAR,
	campaign_id VARCHAR,
	product_id VARCHAR,
	quantity INT,
	item_price DECIMAL(12, 2),
	revenue DECIMAL(12, 2)
);

In [ ]:
CREATE TABLE fact_campaign_daily (
	date_id INT,
	campaign_id VARCHAR,
	channel_id VARCHAR,
	impressions INT,
	clicks INT,
	spend DECIMAL(12, 2),
	orders INT,
	revenue DECIMAL(12, 2)
);

In [ ]:
CREATE TABLE fact_order_items (
	order_item_id VARCHAR,
	order_id VARCHAR,
	product_id VARCHAR,
	product_name VARCHAR,
	quantity INT,
	revenue DECIMAL(12, 2)
)

---

In [ ]:
CREATE TABLE dim_date (
	date_id INT PRIMARY KEY,
	full_date DATE,
	year INT,
	month INT,
	day INT,
	quarter INT,
	month_name VARCHAR(20),	
	day_name VARCHAR(20)
);

In [ ]:
CREATE TABLE dim_customer (
	customer_id VARCHAR PRIMARY KEY,
	first_name VARCHAR,
	last_name VARCHAR,
	email VARCHAR,	
	country VARCHAR,
	customer_type VARCHAR
);

In [ ]:
CREATE TABLE dim_channel (
	channel_id VARCHAR PRIMARY KEY,
	channel_name VARCHAR,
	platform VARCHAR
);

In [ ]:
CREATE TABLE dim_campaign (
	campaign_id VARCHAR PRIMARY KEY,
	channel_id VARCHAR,
	campaign_type VARCHAR
);

In [ ]:
CREATE TABLE dim_product (
	product_id VARCHAR PRIMARY KEY,
	product_name VARCHAR,
	product_category VARCHAR
);

----

In [ ]:
INSERT INTO fact_orders (
	order_id,
	order_date_id,
	customer_id,
	channel_id,
	campaign_id,
	product_id,
	quantity,
	item_price,
	revenue
)

SELECT
	order_id::VARCHAR,
	TO_CHAR(date::DATE, 'YYYYMMDD')::INT AS order_date_id,
	customer_id::VARCHAR,
	channel_id::VARCHAR,
	campaign_id::VARCHAR,
	product_id::VARCHAR,
	quantity::INT,
	item_price::DECIMAL(12, 2),
	revenue::DECIMAL(12, 2)
FROM
	ecommerce_raw_data

In [ ]:
INSERT INTO fact_campaign_daily (
	date_id,
	campaign_id,
	channel_id,
	impressions,
	clicks,
	spend,
	orders,
	revenue
)

SELECT
	TO_CHAR(date::DATE, 'YYYYMMDD')::INT AS date_id,
	campaign_Id::VARCHAR,
	channel_id::VARCHAR,
	impressions::INT,
	clicks::INT,
	spend::DECIMAL(12, 2),
	orders::INT,
	revenue::DECIMAL(12, 2)
FROM
	ecommerce_raw_data;

/*
TABLE: fact_campaign_daily
TYPE: Fact Table (Marketing Campaign Performance)
GRAIN: One row per (date_id, campaign_id, channel_id)

PURPOSE:
--------
This table captures daily performance metrics for marketing campaigns across channels, including:
- impressions: number of ad views
- clicks: number of user interactions
- spend: advertising cost
- orders: attributed purchases
- revenue: revenue generated

DESIGN RATIONALE:
-----------------
1. Fact Table Characteristics:
   - Stores **measurable, additive numeric data**.
   - High cardinality is expected (many campaigns per day, many channels per campaign).
   - Fact tables **allow duplicate date_ids**, because multiple rows can exist for the same date but different campaigns/channels.

2. Primary Key & Duplicates:
   - Original design used `date_id` as PRIMARY KEY.
   - This caused **duplicate key violations** during inserts because the same date may have multiple campaigns and channels.
   - Correct design uses a **composite primary key**:
       PRIMARY KEY (date_id, campaign_id, channel_id)
   - This ensures uniqueness at the **fact table grain** while allowing multiple rows per day.

3. Handling Inserts:
   - PostgreSQL does not have `INSERT IGNORE` like MySQL.
   - Use `ON CONFLICT DO NOTHING` or `ON CONFLICT DO UPDATE` to safely handle duplicate rows or incremental loads.
   - This supports idempotent ETL loads and prevents accidental duplication while respecting the fact table grain.

4. Why duplicates are expected:
   - Each campaign can run on multiple channels each day.
   - The table aggregates performance at **daily × campaign × channel** level.
   - Multiple rows per date are valid and represent distinct business events.

5. Recommended ETL Practices:
   - Always insert new rows with the full grain.
   - Use `ON CONFLICT` to prevent errors for reprocessed or incremental data.
   - Validate source data types (e.g., convert revenue from TEXT to DECIMAL before inserting).
*/

In [ ]:
INSERT INTO fact_order_items (
	order_item_id,
	order_id,
	product_id,
	product_name,
	quantity,
	revenue
)

SELECT
	order_item_id::VARCHAR,
	order_id::VARCHAR,
	product_id::VARCHAR,
	product_name::VARCHAR,
	quantity::INT,
	revenue::DECIMAL(12, 2)
FROM
	ecommerce_raw_data;

---

In [ ]:
INSERT INTO dim_date (
	date_id,
	full_date,
	year,
	month,
	day,
	quarter,
	month_name,
	day_name
)

SELECT DISTINCT
	TO_CHAR(date::DATE, 'YYYYMMDD')::INT AS date_id,
	date::DATE AS full_date,
	EXTRACT(YEAR FROM date::DATE),
	EXTRACT(MONTH FROM date::DATE),
	EXTRACT(DAY FROM date::DATE),
	EXTRACT(QUARTER FROM date::DATE),
	TRIM(TO_CHAR(date::DATE, 'Month')),
	TRIM(TO_CHAR(date::DATE, 'Day'))
FROM ecommerce_raw_data
WHERE date IS NOT NULL;


In [ ]:
INSERT INTO dim_customer (
	customer_id,
	first_name,
	last_name,
	email,
	country,
	customer_type
)

SELECT
	customer_id::VARCHAR,
	SPLIT_PART(customer_name, ' ', 1)::VARCHAR,
	SPLIT_PART(customer_name, ' ', 2)::VARCHAR,
	email::VARCHAR,
	country::VARCHAR,
	customer_type::VARCHAR
FROM
	(
		SELECT *,
			ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY customer_name) AS rn
		FROM 
			ecommerce_raw_data
	) t
	WHERE rn=1

/*
----------------------------------------------------------------------------------------
Purpose: Insert unique customer records into dim_customer dimension table.

Explanation of DISTINCT:
- The DISTINCT keyword applies to all selected columns in the query.
- This means it only removes rows where **all columns have identical values**.
- If customer_id appears in multiple rows with different values in other columns
  (e.g., email, country, customer_type), DISTINCT alone will NOT reduce these
  to a single row per customer_id.
- Because dim_customer.customer_id is a PRIMARY KEY, inserting multiple rows
  with the same customer_id would violate the uniqueness constraint.

Explanation of subquery usage:
- To ensure only **one row per customer_id** is inserted, we use a subquery (derived table)
  with ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY ...).
- The subquery assigns a row number to each row within the same customer_id group.
- Filtering for ROW_NUMBER() = 1 keeps **exactly one row per customer_id**, guaranteeing
  uniqueness for the PRIMARY KEY.
- Note: We are **not trying to preserve all granular transactional data** here.
  The goal is to populate the dimension table with **unique dimensional data**.
  Granularity and multiple transactions per customer are handled in the fact tables.

Summary:
- DISTINCT alone is insufficient for enforcing uniqueness on a single column when
  multiple other columns exist.
- The combination of a subquery with ROW_NUMBER() ensures **deterministic selection**
  of one unique row per customer_id.
- This approach captures the **dimensional uniqueness**, leaving transaction-level
  granularity to fact tables.
----------------------------------------------------------------------------------------
*/

In [ ]:
INSERT INTO dim_channel (
	channel_id,
	channel_name,
	platform
)

SELECT
	channel_id::VARCHAR,
	channel_name::VARCHAR,
	platform_name::VARCHAR AS platform
FROM
	(
		SELECT *,
			ROW_NUMBER() OVER (PARTITION BY channel_id ORDER BY channel_name) AS rn
		FROM 
			ecommerce_raw_data
	) t
	WHERE rn=1

In [ ]:
INSERT INTO dim_campaign (
	campaign_id,
	channel_id,
	campaign_type
)

SELECT
	campaign_id::VARCHAR,
	channel_id::VARCHAR,
	campaign_type::VARCHAR
FROM
	(
		SELECT *,
			ROW_NUMBER() OVER (PARTITION BY campaign_id ORDER BY campaign_type) AS rn
		FROM 
			ecommerce_raw_data
	) t
	WHERE rn=1

In [ ]:
INSERT INTO dim_product(
	product_id,
	product_name,
	product_category
)

SELECT
	product_id::VARCHAR,
	product_name::VARCHAR,
	category::VARCHAR AS product_category
FROM
	(
		SELECT *,
			ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY product_name) AS rn
		FROM 
			ecommerce_raw_data
	) t
	WHERE rn=1

---

In [ ]:
ALTER TABLE fact_orders
ADD CONSTRAINT fk_fact_orders_dim_date
FOREIGN KEY (order_date_id) REFERENCES dim_date(date_id)

In [ ]:
ALTER TABLE fact_orders
ADD CONSTRAINT fk_fact_orders_dim_customer
FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id)

In [ ]:
ALTER TABLE fact_orders
ADD CONSTRAINT fk_fact_orders_dim_channel
FOREIGN KEY (channel_id) REFERENCES dim_channel(channel_id)

In [ ]:
ALTER TABLE fact_orders
ADD CONSTRAINT fk_fact_orders_dim_campaign
FOREIGN KEY (campaign_id) REFERENCES dim_campaign(campaign_id)

In [ ]:
ALTER TABLE fact_orders
ADD CONSTRAINT fk_fact_orders_dim_product
FOREIGN KEY (product_id) REFERENCES dim_product(product_id)

In [ ]:
ALTER TABLE fact_campaign_daily
ADD CONSTRAINT fk_fact_campaign_daily_dim_date
FOREIGN KEY (date_id) REFERENCES dim_date(date_id)

In [ ]:
ALTER TABLE fact_campaign_daily
ADD CONSTRAINT fk_fact_campaign_daily_dim_campaign
FOREIGN KEY (campaign_id) REFERENCES dim_campaign(campaign_id)

In [ ]:
ALTER TABLE fact_campaign_daily
ADD CONSTRAINT fk_fact_campaign_daily_dim_channel
FOREIGN KEY (channel_id) REFERENCES dim_channel(channel_id)

In [ ]:
ALTER TABLE fact_order_items
ADD CONSTRAINT fk_fact_order_items_dim_product
FOREIGN KEY (product_id) REFERENCES dim_product(product_id)